In [1]:
# ============================================
# Shannon's Demon Backtest - Configuration
# ============================================

# 기본 파라미터
INITIAL_CAPITAL = 10_000_000  # 초기 자본: 1,000만원
STOCK_RATIO = 0.5             # 주식 비율: 50%
CASH_RATIO = 0.5              # 현금 비율: 50%

# 거래 비용
TRADING_FEE = 0.00015         # 증권사 수수료: 0.015%
TRADING_TAX = 0.0018          # 증권거래세: 0.18% (매도 시에만)
TOTAL_SELL_COST = TRADING_FEE + TRADING_TAX  # 총 매도 비용: ~0.195%
TOTAL_BUY_COST = TRADING_FEE  # 총 매수 비용: 0.015%

# CMA 금리 (연율)
CMA_ANNUAL_RATE = 0.025       # 연 2.5%
CMA_DAILY_RATE = CMA_ANNUAL_RATE / 365  # 일일 이자율

# 리밸런싱 설정
REBALANCE_DAYS = ['Monday', 'Wednesday', 'Friday']  # 월, 수, 금
REBALANCE_TIME = '10:00'      # 오전 10시

# 백테스팅 기간
START_DATE = '2005-01-01'
END_DATE = '2024-12-31'

# 종목 정보
TICKER_YAHOO = '069500.KS'    # Yahoo Finance 티커
TICKER_FDR = '069500'         # FinanceDataReader 티커
TICKER_NAME = 'KODEX 200'

# 실제 한국 인플레이션 데이터 (연도별, 출처: rateinflation.com)
ACTUAL_INFLATION = {
    2007: 0.0253,
    2008: 0.0467,
    2009: 0.0276,
    2010: 0.0294,
    2011: 0.0403,
    2012: 0.0219,
    2013: 0.0130,
    2014: 0.0127,
    2015: 0.0071,
    2016: 0.0097,
    2017: 0.0194,
    2018: 0.0148,
    2019: 0.0038,
    2020: 0.0054,
    2021: 0.0250,
    2022: 0.0509,
    2023: 0.0359,
    2024: 0.0232,
}

In [2]:
# ============================================
# Shannon's Demon Backtest - Data Loader
# ============================================

import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta

# 위의 config 내용을 여기에 직접 붙여넣거나,
# 같은 셀에서 실행하면 됨

def load_data_yahoo():
    """
    Yahoo Finance에서 KODEX 200 데이터 다운로드
    """
    print(f"Downloading {TICKER_NAME} from Yahoo Finance...")
    print(f"Period: {START_DATE} to {END_DATE}")
    
    data = yf.download(TICKER_YAHOO, start=START_DATE, end=END_DATE, progress=False)
    
    if data.empty:
        raise ValueError("Failed to download data from Yahoo Finance")
    
    print(f"Downloaded {len(data)} trading days")
    return data


def preprocess_data(data):
    """
    데이터 전처리
    """
    df = data.copy()
    
    # 필요한 컬럼만 추출 (종가 기준)
    if 'Adj Close' in df.columns:
        df = df[['Adj Close', 'Close', 'Volume']].copy()
        df.columns = ['adj_close', 'close', 'volume']
    else:
        df = df[['Close', 'Volume']].copy()
        df.columns = ['close', 'volume']
        df['adj_close'] = df['close']
    
    # 결측치 처리
    df = df.ffill()
    df.dropna(inplace=True)
    
    # 요일 정보 추가
    df['day_of_week'] = df.index.day_name()

    
    # 일간 수익률 계산
    df['daily_return'] = df['adj_close'].pct_change()
    
    print(f"\nData preprocessing completed")
    print(f"Final dataset: {len(df)} trading days")
    print(f"Date range: {df.index[0].strftime('%Y-%m-%d')} to {df.index[-1].strftime('%Y-%m-%d')}")
    
    return df


def get_data_summary(df):
    """
    데이터 요약 통계 출력
    """
    print("\n" + "="*50)
    print("DATA SUMMARY")
    print("="*50)
    
    print(f"\n[Basic Info]")
    print(f"Total trading days: {len(df)}")
    print(f"Start date: {df.index[0].strftime('%Y-%m-%d')}")
    print(f"End date: {df.index[-1].strftime('%Y-%m-%d')}")
    print(f"Years covered: {(df.index[-1] - df.index[0]).days / 365:.1f} years")
    
    print(f"\n[Price Info]")
    print(f"Starting price: {df['adj_close'].iloc[0]:,.0f} KRW")
    print(f"Ending price: {df['adj_close'].iloc[-1]:,.0f} KRW")
    print(f"Min price: {df['adj_close'].min():,.0f} KRW")
    print(f"Max price: {df['adj_close'].max():,.0f} KRW")
    
    total_return = (df['adj_close'].iloc[-1] / df['adj_close'].iloc[0] - 1) * 100
    years = (df.index[-1] - df.index[0]).days / 365
    cagr = ((df['adj_close'].iloc[-1] / df['adj_close'].iloc[0]) ** (1/years) - 1) * 100
    
    print(f"\n[Buy & Hold Performance]")
    print(f"Total return: {total_return:.2f}%")
    print(f"CAGR: {cagr:.2f}%")
    print(f"Daily volatility: {df['daily_return'].std() * 100:.2f}%")
    print(f"Annual volatility: {df['daily_return'].std() * np.sqrt(252) * 100:.2f}%")
    
    print("="*50)

In [3]:
raw_data = load_data_yahoo()
df = preprocess_data(raw_data)
get_data_summary(df)

df.head(10)

Period: 2005-01-01 to 2024-12-31
Downloaded 3853 trading days

Data preprocessing completed
Final dataset: 3853 trading days
Date range: 2007-01-29 to 2024-12-30

DATA SUMMARY

[Basic Info]
Total trading days: 3853
Start date: 2007-01-29
End date: 2024-12-30
Years covered: 17.9 years

[Price Info]
Starting price: 13,423 KRW
Ending price: 31,351 KRW
Min price: 12,667 KRW
Max price: 40,223 KRW

[Buy & Hold Performance]
Total return: 133.57%
CAGR: 4.84%
Daily volatility: 1.11%
Annual volatility: 17.68%


,close,volume,adj_close,day_of_week,daily_return
Date,,,,,
2007-01-29,13422.593750,433267,13422.593750,Monday,NaN
2007-01-30,13474.679688,205208,13474.679688,Tuesday,0.003880
2007-01-31,13403.996094,553305,13403.996094,Wednesday,-0.005246
2007-02-01,13597.449219,515988,13597.449219,Thursday,0.014432
2007-02-02,13887.627930,1031912,13887.627930,Friday,0.021341
2007-02-05,13976.910156,1196916,13976.910156,Monday,0.006429
2007-02-06,14058.753906,557476,14058.753906,Tuesday,0.005856
2007-02-07,14006.674805,677733,14006.674805,Wednesday,-0.003704
2009-04-17,12924.083008,1899707,12924.083008,Friday,-0.077291


In [4]:
# ============================================
# Shannon's Demon Backtest - Engine
# ============================================

import pandas as pd
import numpy as np

def run_backtest(df, 
                 initial_capital=10_000_000,
                 stock_ratio=0.5,
                 rebalance_days=['Monday', 'Wednesday', 'Friday'],
                 trading_fee=0.00015,
                 trading_tax=0.0018,
                 cma_annual_rate=0.025):
    """
    Shannon's Demon 백테스팅 실행
    
    Parameters:
    -----------
    df : DataFrame
        전처리된 가격 데이터 (adj_close, day_of_week 필요)
    initial_capital : float
        초기 자본금
    stock_ratio : float
        목표 주식 비율 (0.5 = 50%)
    rebalance_days : list
        리밸런싱 요일
    trading_fee : float
        증권사 수수료 (매수/매도 모두)
    trading_tax : float
        증권거래세 (매도 시에만)
    cma_annual_rate : float
        CMA 연이자율
    
    Returns:
    --------
    result_df : DataFrame
        일별 포트폴리오 상태
    trades : list
        거래 내역
    """
    
    # 일일 CMA 이자율
    cma_daily_rate = cma_annual_rate / 365
    
    # 결과 저장용
    records = []
    trades = []
    
    # 초기 설정
    cash = initial_capital * (1 - stock_ratio)
    stock_value = initial_capital * stock_ratio
    first_price = df['adj_close'].iloc[0]
    shares = stock_value / first_price  # 보유 주식 수
    
    # 초기 매수 비용
    initial_buy_cost = stock_value * trading_fee
    cash -= initial_buy_cost
    
    trades.append({
        'date': df.index[0],
        'action': 'INITIAL_BUY',
        'shares': shares,
        'price': first_price,
        'amount': stock_value,
        'cost': initial_buy_cost
    })
    
    # 매일 시뮬레이션
    for i, (date, row) in enumerate(df.iterrows()):
        price = row['adj_close']
        day_of_week = row['day_of_week']
        
        # 현재 주식 가치 업데이트
        stock_value = shares * price
        
        # CMA 이자 적용 (매일)
       # CMA 이자 적용 (실제 경과일 기준)
        if i > 0:
            prev_date = df.index[i-1]
            days_passed = (date - prev_date).days
            cash_interest = cash * cma_daily_rate * days_passed
            cash += cash_interest
        else:
            cash_interest = 0
        
        # 총 포트폴리오 가치
        total_value = cash + stock_value
        current_stock_ratio = stock_value / total_value
        
        # 리밸런싱 여부 확인
        rebalanced = False
        trade_cost = 0
        
        if day_of_week in rebalance_days and i > 0:
            # 목표 대비 괴리 계산
            target_stock_value = total_value * stock_ratio
            diff = stock_value - target_stock_value
            
            if abs(diff) > 0:  # 리밸런싱 필요
                rebalanced = True
                
                if diff > 0:
                    # 주식 비중 높음 → 매도
                    sell_amount = diff
                    sell_shares = sell_amount / price
                    sell_cost = sell_amount * (trading_fee + trading_tax)
                    
                    shares -= sell_shares
                    cash += sell_amount - sell_cost
                    trade_cost = sell_cost
                    
                    trades.append({
                        'date': date,
                        'action': 'SELL',
                        'shares': sell_shares,
                        'price': price,
                        'amount': sell_amount,
                        'cost': sell_cost
                    })
                    
                else:
                    # 주식 비중 낮음 → 매수
                    buy_amount = abs(diff)
                    buy_cost = buy_amount * trading_fee
                    buy_shares = buy_amount / price
                    
                    shares += buy_shares
                    cash -= buy_amount + buy_cost
                    trade_cost = buy_cost
                    
                    trades.append({
                        'date': date,
                        'action': 'BUY',
                        'shares': buy_shares,
                        'price': price,
                        'amount': buy_amount,
                        'cost': buy_cost
                    })
                
                # 리밸런싱 후 가치 재계산
                stock_value = shares * price
                total_value = cash + stock_value
                current_stock_ratio = stock_value / total_value
        
        # 기록 저장
        records.append({
            'date': date,
            'price': price,
            'shares': shares,
            'stock_value': stock_value,
            'cash': cash,
            'total_value': total_value,
            'stock_ratio': current_stock_ratio,
            'rebalanced': rebalanced,
            'trade_cost': trade_cost,
            'cash_interest': cash_interest
        })
    
    # DataFrame 변환
    result_df = pd.DataFrame(records)
    result_df.set_index('date', inplace=True)
    
    # 일간 수익률 계산
    result_df['daily_return'] = result_df['total_value'].pct_change()
    
    return result_df, trades


def run_buy_and_hold(df, initial_capital=10_000_000, trading_fee=0.00015):
    """
    Buy & Hold 전략 (비교용)
    """
    records = []
    
    first_price = df['adj_close'].iloc[0]
    buy_cost = initial_capital * trading_fee
    shares = (initial_capital - buy_cost) / first_price
    
    for date, row in df.iterrows():
        price = row['adj_close']
        total_value = shares * price
        
        records.append({
            'date': date,
            'price': price,
            'total_value': total_value
        })
    
    result_df = pd.DataFrame(records)
    result_df.set_index('date', inplace=True)
    result_df['daily_return'] = result_df['total_value'].pct_change()
    
    return result_df


def calculate_metrics(result_df, strategy_name="Strategy"):
    """
    성과 지표 계산
    """
    total_value = result_df['total_value']
    daily_returns = result_df['daily_return'].dropna()
    
    # 기본 수익률
    total_return = (total_value.iloc[-1] / total_value.iloc[0] - 1) * 100
    
    # CAGR
    years = (result_df.index[-1] - result_df.index[0]).days / 365
    cagr = ((total_value.iloc[-1] / total_value.iloc[0]) ** (1/years) - 1) * 100
    
    # 변동성
    daily_vol = daily_returns.std()
    annual_vol = daily_vol * np.sqrt(252) * 100
    
    # Sharpe Ratio (무위험 수익률 2.5% 가정)
    risk_free_rate = 0.025
    excess_return = cagr / 100 - risk_free_rate
    sharpe = excess_return / (annual_vol / 100) if annual_vol > 0 else 0
    
    # MDD (Maximum Drawdown)
    cummax = total_value.cummax()
    drawdown = (total_value - cummax) / cummax
    mdd = drawdown.min() * 100
    
    # MDD 기간
    mdd_end_idx = drawdown.idxmin()
    mdd_start_idx = total_value[:mdd_end_idx].idxmax()
    
    # 회복 기간 계산
    recovery_idx = total_value[mdd_end_idx:][total_value[mdd_end_idx:] >= cummax[mdd_end_idx]].index
    if len(recovery_idx) > 0:
        recovery_date = recovery_idx[0]
        recovery_days = (recovery_date - mdd_end_idx).days
    else:
        recovery_date = None
        recovery_days = None
    
    metrics = {
        'strategy': strategy_name,
        'total_return': total_return,
        'cagr': cagr,
        'annual_volatility': annual_vol,
        'sharpe_ratio': sharpe,
        'mdd': mdd,
        'mdd_start': mdd_start_idx,
        'mdd_end': mdd_end_idx,
        'recovery_days': recovery_days,
        'final_value': total_value.iloc[-1]
    }
    
    return metrics


def print_metrics(metrics):
    """
    성과 지표 출력
    """
    print(f"\n{'='*50}")
    print(f" {metrics['strategy']} Performance")
    print(f"{'='*50}")
    print(f"Total Return:      {metrics['total_return']:>10.2f}%")
    print(f"CAGR:              {metrics['cagr']:>10.2f}%")
    print(f"Annual Volatility: {metrics['annual_volatility']:>10.2f}%")
    print(f"Sharpe Ratio:      {metrics['sharpe_ratio']:>10.2f}")
    print(f"Max Drawdown:      {metrics['mdd']:>10.2f}%")
    print(f"Final Value:       {metrics['final_value']:>13,.0f} KRW")
    print(f"{'='*50}")


def compare_strategies(shannon_metrics, bh_metrics):
    """
    두 전략 비교
    """
    print(f"\n{'='*60}")
    print(f" Strategy Comparison")
    print(f"{'='*60}")
    print(f"{'Metric':<20} {'Shannon Demon':>15} {'Buy & Hold':>15}")
    print(f"{'-'*60}")
    print(f"{'Total Return':<20} {shannon_metrics['total_return']:>14.2f}% {bh_metrics['total_return']:>14.2f}%")
    print(f"{'CAGR':<20} {shannon_metrics['cagr']:>14.2f}% {bh_metrics['cagr']:>14.2f}%")
    print(f"{'Annual Volatility':<20} {shannon_metrics['annual_volatility']:>14.2f}% {bh_metrics['annual_volatility']:>14.2f}%")
    print(f"{'Sharpe Ratio':<20} {shannon_metrics['sharpe_ratio']:>15.2f} {bh_metrics['sharpe_ratio']:>15.2f}")
    print(f"{'Max Drawdown':<20} {shannon_metrics['mdd']:>14.2f}% {bh_metrics['mdd']:>14.2f}%")
    print(f"{'Final Value (KRW)':<20} {shannon_metrics['final_value']:>15,.0f} {bh_metrics['final_value']:>15,.0f}")
    print(f"{'='*60}")
    
    # 승자 판정
    print(f"\n[Winner by Metric]")
    print(f"  CAGR:       {'Shannon Demon ✓' if shannon_metrics['cagr'] > bh_metrics['cagr'] else 'Buy & Hold ✓'}")
    print(f"  Sharpe:     {'Shannon Demon ✓' if shannon_metrics['sharpe_ratio'] > bh_metrics['sharpe_ratio'] else 'Buy & Hold ✓'}")
    print(f"  MDD:        {'Shannon Demon ✓' if shannon_metrics['mdd'] > bh_metrics['mdd'] else 'Buy & Hold ✓'}")

In [5]:
# 백테스팅 실행
shannon_result, trades = run_backtest(
    df,
    initial_capital=10_000_000,
    stock_ratio=0.5,
    rebalance_days=['Monday', 'Wednesday', 'Friday']
)

# Buy & Hold 실행
bh_result = run_buy_and_hold(df, initial_capital=10_000_000)

# 성과 지표 계산
shannon_metrics = calculate_metrics(shannon_result, "Shannon's Demon")
bh_metrics = calculate_metrics(bh_result, "Buy & Hold")

# 결과 출력
print_metrics(shannon_metrics)
print_metrics(bh_metrics)
compare_strategies(shannon_metrics, bh_metrics)

# 거래 횟수 확인
print(f"\nTotal trades: {len(trades)}")
print(f"Total trading cost: {sum(t['cost'] for t in trades):,.0f} KRW")


 Shannon's Demon Performance
Total Return:          102.13%
CAGR:                    4.00%
Annual Volatility:       8.78%
Sharpe Ratio:            0.17
Max Drawdown:          -18.72%
Final Value:          20,211,628 KRW

 Buy & Hold Performance
Total Return:          133.57%
CAGR:                    4.84%
Annual Volatility:      17.68%
Sharpe Ratio:            0.13
Max Drawdown:          -38.08%
Final Value:          23,353,444 KRW

 Strategy Comparison
Metric                 Shannon Demon      Buy & Hold
------------------------------------------------------------
Total Return                 102.13%         133.57%
CAGR                           4.00%           4.84%
Annual Volatility              8.78%          17.68%
Sharpe Ratio                    0.17            0.13
Max Drawdown                 -18.72%         -38.08%
Final Value (KRW)         20,211,628      23,353,444

[Winner by Metric]
  CAGR:       Buy & Hold ✓
  Sharpe:     Shannon Demon ✓
  MDD:        Shannon Demon ✓

T

In [6]:
# ============================================
# Three Strategy Comparison
# ============================================

def run_pure_saving(df, initial_capital=10_000_000, annual_rate=0.025):
    """
    순수 저축 전략 - 실제 경과일 기준 이자 계산
    """
    daily_rate = annual_rate / 365
    records = []
    
    balance = initial_capital
    prev_date = None
    
    for date, row in df.iterrows():
        if prev_date is not None:
            # 실제 경과 일수만큼 이자 적용
            days_passed = (date - prev_date).days
            interest = balance * daily_rate * days_passed
            balance += interest
        else:
            interest = 0
        
        prev_date = date
        
        records.append({
            'date': date,
            'total_value': balance,
            'interest': interest
        })
    
    result_df = pd.DataFrame(records)
    result_df.set_index('date', inplace=True)
    result_df['daily_return'] = result_df['total_value'].pct_change()
    
    return result_df


def run_all_strategies(df, initial_capital=10_000_000):
    """
    세 가지 전략 모두 실행
    """
    print("Running Shannon's Demon (50/50)...")
    shannon_result, trades = run_backtest(df, initial_capital=initial_capital)
    
    print("Running Buy & Hold...")
    bh_result = run_buy_and_hold(df, initial_capital=initial_capital)
    
    print("Running Pure Saving...")
    saving_result = run_pure_saving(df, initial_capital=initial_capital)
    
    return shannon_result, bh_result, saving_result, trades


def calculate_real_return(nominal_cagr, inflation_rate=0.025):
    """
    실질 수익률 계산 (Fisher equation)
    """
    real_return = ((1 + nominal_cagr/100) / (1 + inflation_rate) - 1) * 100
    return real_return

def calculate_actual_real_return(result_df, inflation_data=ACTUAL_INFLATION):
    """
    실제 인플레이션 데이터를 사용한 실질 수익률 계산
    """
    yearly_values = result_df['total_value'].resample('YE').last()
    
    cumulative_inflation = 1.0
    
    for i, (date, value) in enumerate(yearly_values.items()):
        if i == 0:
            continue
        year = date.year
        if year in inflation_data:
            cumulative_inflation *= (1 + inflation_data[year])
    
    initial_value = yearly_values.iloc[0]
    final_nominal = yearly_values.iloc[-1]
    final_real = final_nominal / cumulative_inflation
    years = len(yearly_values) - 1
    
    real_cagr = ((final_real / initial_value) ** (1 / years) - 1) * 100
    
    return {
        'real_cagr': real_cagr,
        'final_real_value': final_real,
        'cumulative_inflation': (cumulative_inflation - 1) * 100,
        'avg_annual_inflation': (cumulative_inflation ** (1 / years) - 1) * 100
    }

def compare_three_strategies(shannon_result, bh_result, saving_result, assumed_inflation=0.025):
    """
    세 전략 비교 분석 (실제 + 가정 인플레이션 모두 표시)
    """
    # 각 전략 지표 계산
    shannon_metrics = calculate_metrics(shannon_result, "Shannon's Demon")
    bh_metrics = calculate_metrics(bh_result, "Buy & Hold")
    saving_metrics = calculate_metrics(saving_result, "Pure Saving")
    
    # 가정 인플레이션 기준 실질 수익률
    shannon_assumed = calculate_real_return(shannon_metrics['cagr'], assumed_inflation)
    bh_assumed = calculate_real_return(bh_metrics['cagr'], assumed_inflation)
    saving_assumed = calculate_real_return(saving_metrics['cagr'], assumed_inflation)
    
    # 실제 인플레이션 기준 실질 수익률
    shannon_actual = calculate_actual_real_return(shannon_result)
    bh_actual = calculate_actual_real_return(bh_result)
    saving_actual = calculate_actual_real_return(saving_result)
    
    # 출력
    print(f"\n{'='*80}")
    print(f"  Three Strategy Comparison")
    print(f"{'='*80}")
    print(f"{'Metric':<28} {'Shannon Demon':>15} {'Buy & Hold':>15} {'Pure Saving':>15}")
    print(f"{'-'*80}")
    print(f"{'Total Return':<28} {shannon_metrics['total_return']:>14.2f}% {bh_metrics['total_return']:>14.2f}% {saving_metrics['total_return']:>14.2f}%")
    print(f"{'CAGR (Nominal)':<28} {shannon_metrics['cagr']:>14.2f}% {bh_metrics['cagr']:>14.2f}% {saving_metrics['cagr']:>14.2f}%")
    print(f"{'CAGR (Real-Actual)':<28} {shannon_actual['real_cagr']:>14.2f}% {bh_actual['real_cagr']:>14.2f}% {saving_actual['real_cagr']:>14.2f}%")
    print(f"{'CAGR (Real-Assumed 2.5%)':<28} {shannon_assumed:>14.2f}% {bh_assumed:>14.2f}% {saving_assumed:>14.2f}%")
    print(f"{'Annual Volatility':<28} {shannon_metrics['annual_volatility']:>14.2f}% {bh_metrics['annual_volatility']:>14.2f}% {saving_metrics['annual_volatility']:>14.2f}%")
    print(f"{'Sharpe Ratio':<28} {shannon_metrics['sharpe_ratio']:>15.2f} {bh_metrics['sharpe_ratio']:>15.2f} {saving_metrics['sharpe_ratio']:>15.2f}")
    print(f"{'Max Drawdown':<28} {shannon_metrics['mdd']:>14.2f}% {bh_metrics['mdd']:>14.2f}% {saving_metrics['mdd']:>14.2f}%")
    print(f"{'Final Value (KRW)':<28} {shannon_metrics['final_value']:>15,.0f} {bh_metrics['final_value']:>15,.0f} {saving_metrics['final_value']:>15,.0f}")
    print(f"{'Final Real Value (KRW)':<28} {shannon_actual['final_real_value']:>15,.0f} {bh_actual['final_real_value']:>15,.0f} {saving_actual['final_real_value']:>15,.0f}")
    print(f"{'='*80}")
    
    # 인플레이션 정보
    print(f"\n[Inflation Data]")
    print(f"  Actual cumulative (2007-2024): {shannon_actual['cumulative_inflation']:.2f}%")
    print(f"  Actual avg annual: {shannon_actual['avg_annual_inflation']:.2f}%")
    print(f"  Assumed: {assumed_inflation*100:.2f}%")
    
    # 저축 대비 초과 수익 (명목)
    shannon_excess = shannon_metrics['final_value'] - saving_metrics['final_value']
    bh_excess = bh_metrics['final_value'] - saving_metrics['final_value']
    
    print(f"\n[Excess Return vs Pure Saving (Nominal)]")
    print(f"  Shannon's Demon: {shannon_excess:>+15,.0f} KRW ({shannon_excess/saving_metrics['final_value']*100:>+.2f}%)")
    print(f"  Buy & Hold:      {bh_excess:>+15,.0f} KRW ({bh_excess/saving_metrics['final_value']*100:>+.2f}%)")
    
    # 저축 대비 초과 수익 (실질)
    shannon_real_excess = shannon_actual['final_real_value'] - saving_actual['final_real_value']
    bh_real_excess = bh_actual['final_real_value'] - saving_actual['final_real_value']
    
    print(f"\n[Excess Return vs Pure Saving (Real)]")
    print(f"  Shannon's Demon: {shannon_real_excess:>+15,.0f} KRW ({shannon_real_excess/saving_actual['final_real_value']*100:>+.2f}%)")
    print(f"  Buy & Hold:      {bh_real_excess:>+15,.0f} KRW ({bh_real_excess/saving_actual['final_real_value']*100:>+.2f}%)")
    
    # 리스크 대비 수익
    print(f"\n[Risk-Adjusted Analysis (Actual Inflation)]")
    print(f"  Shannon's Demon: {shannon_actual['real_cagr']:.2f}% real return with {abs(shannon_metrics['mdd']):.1f}% max risk")
    print(f"  Buy & Hold:      {bh_actual['real_cagr']:.2f}% real return with {abs(bh_metrics['mdd']):.1f}% max risk")
    print(f"  Pure Saving:     {saving_actual['real_cagr']:.2f}% real return with {abs(saving_metrics['mdd']):.1f}% max risk")
    
    return shannon_metrics, bh_metrics, saving_metrics

# ============================================
# Rebalancing Frequency Comparison
# ============================================

REBALANCE_WEEKLY_1 = ['Wednesday']  # 주 1회
REBALANCE_WEEKLY_3 = ['Monday', 'Wednesday', 'Friday']  # 주 3회
REBALANCE_WEEKLY_5 = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']  # 주 5회

def run_shannon_with_frequency(df, rebalance_days, initial_capital=10_000_000):
    """
    지정된 리밸런싱 빈도로 Shannon's Demon 실행
    """
    # 리밸런싱 대상일 설정
    df_copy = df.copy()
    df_copy['is_rebalance_day'] = df_copy['day_of_week'].isin(rebalance_days)
    
    records = []
    trades = []
    
    # 초기 포지션
    stock_value = initial_capital * STOCK_RATIO
    cash_value = initial_capital * CASH_RATIO
    
    first_price = df_copy['adj_close'].iloc[0]
    shares = stock_value / first_price
    stock_value = shares * first_price
    
    buy_cost = stock_value * TOTAL_BUY_COST
    cash_value -= buy_cost
    
    trades.append({
        'date': df_copy.index[0],
        'action': 'INITIAL_BUY',
        'shares': shares,
        'price': first_price,
        'cost': buy_cost
    })
    
    prev_date = None
    
    for date, row in df_copy.iterrows():
        price = row['adj_close']
        
        # CMA 이자 (실제 경과일 기준)
        if prev_date is not None:
            days_passed = (date - prev_date).days
            interest = cash_value * CMA_DAILY_RATE * days_passed
            cash_value += interest
        
        stock_value = shares * price
        total_value = stock_value + cash_value
        
        # 리밸런싱
        if row['is_rebalance_day'] and prev_date is not None:
            target_stock_value = total_value * STOCK_RATIO
            diff = target_stock_value - stock_value
            
            if abs(diff) > 1000:
                if diff > 0:  # 매수
                    shares_to_buy = diff / price
                    cost = diff * TOTAL_BUY_COST
                    shares += shares_to_buy
                    cash_value -= (diff + cost)
                    trades.append({'date': date, 'action': 'BUY', 'shares': shares_to_buy, 'price': price, 'cost': cost})
                else:  # 매도
                    shares_to_sell = abs(diff) / price
                    cost = abs(diff) * TOTAL_SELL_COST
                    shares -= shares_to_sell
                    cash_value += (abs(diff) - cost)
                    trades.append({'date': date, 'action': 'SELL', 'shares': shares_to_sell, 'price': price, 'cost': cost})
                
                stock_value = shares * price
                total_value = stock_value + cash_value
        
        records.append({
            'date': date,
            'price': price,
            'shares': shares,
            'stock_value': stock_value,
            'cash_value': cash_value,
            'total_value': total_value
        })
        
        prev_date = date
    
    result_df = pd.DataFrame(records)
    result_df.set_index('date', inplace=True)
    result_df['daily_return'] = result_df['total_value'].pct_change()
    
    return result_df, trades


def compare_rebalancing_frequencies(df, initial_capital=10_000_000):
    """
    리밸런싱 빈도별 성과 비교
    """
    frequencies = {
        'Weekly 1x (Wed)': REBALANCE_WEEKLY_1,
        'Weekly 3x (M/W/F)': REBALANCE_WEEKLY_3,
        'Weekly 5x (Daily)': REBALANCE_WEEKLY_5
    }
    
    results = {}
    
    for name, rebalance_days in frequencies.items():
        print(f"Running {name}...")
        result_df, trades = run_shannon_with_frequency(df, rebalance_days, initial_capital)
        metrics = calculate_metrics(result_df, name)
        real_return = calculate_actual_real_return(result_df)
        
        results[name] = {
            'df': result_df,
            'trades': trades,
            'metrics': metrics,
            'real': real_return,
            'total_cost': sum(t['cost'] for t in trades)
        }
    
    # 결과 출력
    print(f"\n{'='*85}")
    print(f"  Rebalancing Frequency Comparison")
    print(f"{'='*85}")
    print(f"{'Metric':<25} {'Weekly 1x':>18} {'Weekly 3x':>18} {'Weekly 5x':>18}")
    print(f"{'-'*85}")
    
    names = list(frequencies.keys())
    
    print(f"{'Total Return':<25} {results[names[0]]['metrics']['total_return']:>17.2f}% {results[names[1]]['metrics']['total_return']:>17.2f}% {results[names[2]]['metrics']['total_return']:>17.2f}%")
    print(f"{'CAGR (Nominal)':<25} {results[names[0]]['metrics']['cagr']:>17.2f}% {results[names[1]]['metrics']['cagr']:>17.2f}% {results[names[2]]['metrics']['cagr']:>17.2f}%")
    print(f"{'CAGR (Real-Actual)':<25} {results[names[0]]['real']['real_cagr']:>17.2f}% {results[names[1]]['real']['real_cagr']:>17.2f}% {results[names[2]]['real']['real_cagr']:>17.2f}%")
    print(f"{'Annual Volatility':<25} {results[names[0]]['metrics']['annual_volatility']:>17.2f}% {results[names[1]]['metrics']['annual_volatility']:>17.2f}% {results[names[2]]['metrics']['annual_volatility']:>17.2f}%")
    print(f"{'Sharpe Ratio':<25} {results[names[0]]['metrics']['sharpe_ratio']:>18.2f} {results[names[1]]['metrics']['sharpe_ratio']:>18.2f} {results[names[2]]['metrics']['sharpe_ratio']:>18.2f}")
    print(f"{'Max Drawdown':<25} {results[names[0]]['metrics']['mdd']:>17.2f}% {results[names[1]]['metrics']['mdd']:>17.2f}% {results[names[2]]['metrics']['mdd']:>17.2f}%")
    print(f"{'Final Value (KRW)':<25} {results[names[0]]['metrics']['final_value']:>17,.0f} {results[names[1]]['metrics']['final_value']:>17,.0f} {results[names[2]]['metrics']['final_value']:>17,.0f}")
    print(f"{'Final Real Value (KRW)':<25} {results[names[0]]['real']['final_real_value']:>17,.0f} {results[names[1]]['real']['final_real_value']:>17,.0f} {results[names[2]]['real']['final_real_value']:>17,.0f}")
    print(f"{'-'*85}")
    print(f"{'Total Trades':<25} {len(results[names[0]]['trades']):>18,} {len(results[names[1]]['trades']):>18,} {len(results[names[2]]['trades']):>18,}")
    print(f"{'Total Trading Cost':<25} {results[names[0]]['total_cost']:>17,.0f} {results[names[1]]['total_cost']:>17,.0f} {results[names[2]]['total_cost']:>17,.0f}")
    print(f"{'='*85}")
    
    # 순이익 비교 (수익 - 거래비용)
    print(f"\n[Net Profit Analysis]")
    for name in names:
        gross = results[name]['metrics']['final_value'] - initial_capital
        cost = results[name]['total_cost']
        net = gross
        print(f"  {name}: Gross {gross:+,.0f} | Cost {cost:,.0f} | Net {net:+,.0f}")
    
    return results

In [7]:
# 세 전략 실행
shannon_result, bh_result, saving_result, trades = run_all_strategies(df)

# 비교 분석 (인플레이션 2.5% 가정)
shannon_m, bh_m, saving_m = compare_three_strategies(
    shannon_result, 
    bh_result, 
    saving_result,
    assumed_inflation =0.025
)

print(f"\nTotal trades (Shannon): {len(trades)}")
print(f"Total trading cost: {sum(t['cost'] for t in trades):,.0f} KRW")

Running Shannon's Demon (50/50)...
Running Buy & Hold...
Running Pure Saving...

  Three Strategy Comparison
Metric                         Shannon Demon      Buy & Hold     Pure Saving
--------------------------------------------------------------------------------
Total Return                         102.13%         133.57%          56.33%
CAGR (Nominal)                         4.00%           4.84%           2.52%
CAGR (Real-Actual)                     1.79%           2.53%           0.38%
CAGR (Real-Assumed 2.5%)               1.47%           2.29%           0.02%
Annual Volatility                      8.78%          17.68%           1.40%
Sharpe Ratio                            0.17            0.13            0.02
Max Drawdown                         -18.72%         -38.08%           0.00%
Final Value (KRW)                 20,211,628      23,353,444      15,633,185
Final Real Value (KRW)            13,809,369      15,955,979      10,681,199

[Inflation Data]
  Actual cumulative (2

In [8]:
# 리밸런싱 빈도 비교 실행
freq_results = compare_rebalancing_frequencies(df)

Running Weekly 1x (Wed)...
Running Weekly 3x (M/W/F)...
Running Weekly 5x (Daily)...

  Rebalancing Frequency Comparison
Metric                             Weekly 1x          Weekly 3x          Weekly 5x
-------------------------------------------------------------------------------------
Total Return                         102.09%            102.13%            101.54%
CAGR (Nominal)                         4.00%              4.00%              3.99%
CAGR (Real-Actual)                     1.79%              1.79%              1.77%
Annual Volatility                      8.75%              8.78%              8.78%
Sharpe Ratio                            0.17               0.17               0.17
Max Drawdown                         -18.49%            -18.72%            -18.77%
Final Value (KRW)                20,207,897        20,211,601        20,152,302
Final Real Value (KRW)           13,806,820        13,809,350        13,768,835
----------------------------------------------------